In [ ]:
!pip install kafka-python

In [1]:
from time import time
from kafka import KafkaProducer
import pandas as pd
import json
import pyspark
from pyspark.sql import SparkSession

import pyspark.sql.functions  as F 
from pyspark.sql import types





In [2]:




def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

In [3]:
! docker exec -it redpanda-1 rpk topic create test-topic


TOPIC       STATUS
test-topic  TOPIC_ALREADY_EXISTS: Topic with this name already exists.


In [4]:
import time  # Import the time module from the Python standard library

t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)

producer.flush()

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')


Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}
took 0.52 seconds


In [5]:
!docker exec -it redpanda-1 rpk topic consume test-topic

{
  "topic": "test-topic",
  "value": "{\"number\": 0}",
  "timestamp": 1711310802877,
  "partition": 0,
  "offset": 0
}
{
  "topic": "test-topic",
  "value": "{\"number\": 1}",
  "timestamp": 1711310802930,
  "partition": 0,
  "offset": 1
}
{
  "topic": "test-topic",
  "value": "{\"number\": 2}",
  "timestamp": 1711310802980,
  "partition": 0,
  "offset": 2
}
{
  "topic": "test-topic",
  "value": "{\"number\": 3}",
  "timestamp": 1711310803031,
  "partition": 0,
  "offset": 3
}
{
  "topic": "test-topic",
  "value": "{\"number\": 4}",
  "timestamp": 1711310803082,
  "partition": 0,
  "offset": 4
}
{
  "topic": "test-topic",
  "value": "{\"number\": 5}",
  "timestamp": 1711310803134,
  "partition": 0,
  "offset": 5
}
{
  "topic": "test-topic",
  "value": "{\"number\": 6}",
  "timestamp": 1711310803186,
  "partition": 0,
  "offset": 6
}
{
  "topic": "test-topic",
  "value": "{\"number\": 7}",
  "timestamp": 1711310803236,
  "partition": 0,
  "offset": 7
}
{
  "topic": "test-topic",
  "va

In [6]:
import pandas as pd

# Read the green CSV file
df_green = pd.read_csv('/home/zoomcamp/kafka_week/green_tripdata_2019-10.csv.gz', usecols=[
    'lpep_pickup_datetime',
    'lpep_dropoff_datetime',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'tip_amount'
])

# Iterate over the records in the dataframe
for row in df_green.head().itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    # Print the dictionary representation of the row for debugging
    print(row_dict)
    # TODO: Implement sending the data here
    # You can send the data to your desired destination, e.g., Kafka, database, etc.


{'lpep_pickup_datetime': '2019-10-01 00:26:02', 'lpep_dropoff_datetime': '2019-10-01 00:39:58', 'PULocationID': 112, 'DOLocationID': 196, 'passenger_count': 1.0, 'trip_distance': 5.88, 'tip_amount': 0.0}
{'lpep_pickup_datetime': '2019-10-01 00:18:11', 'lpep_dropoff_datetime': '2019-10-01 00:22:38', 'PULocationID': 43, 'DOLocationID': 263, 'passenger_count': 1.0, 'trip_distance': 0.8, 'tip_amount': 0.0}
{'lpep_pickup_datetime': '2019-10-01 00:09:31', 'lpep_dropoff_datetime': '2019-10-01 00:24:47', 'PULocationID': 255, 'DOLocationID': 228, 'passenger_count': 2.0, 'trip_distance': 7.5, 'tip_amount': 0.0}
{'lpep_pickup_datetime': '2019-10-01 00:37:40', 'lpep_dropoff_datetime': '2019-10-01 00:41:49', 'PULocationID': 181, 'DOLocationID': 181, 'passenger_count': 1.0, 'trip_distance': 0.9, 'tip_amount': 0.0}
{'lpep_pickup_datetime': '2019-10-01 00:08:13', 'lpep_dropoff_datetime': '2019-10-01 00:17:56', 'PULocationID': 97, 'DOLocationID': 188, 'passenger_count': 1.0, 'trip_distance': 2.52, 'tip

In [8]:
!docker exec -it redpanda-1 rpk topic create green-trips

#!docker exec -it redpanda-1 rpk topic delete green-trips

TOPIC        STATUS
green-trips  OK


In [9]:


def json_serializer(data):
    return json.dumps(data).encode('utf-8')

# Define the server address
server = 'localhost:9092'

# Initialize the KafkaProducer
producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

# Define the topic name
topic_name = 'green-trips'

# Record the start time
start_time = time.time()
# Read data from the CSV file with specific columns
df_green = pd.read_csv('/home/zoomcamp/kafka_week/green_tripdata_2019-10.csv.gz', usecols=[
    'lpep_pickup_datetime',
    'lpep_dropoff_datetime',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'tip_amount'
])

# Convert DataFrame to a list of dictionaries with column names
data_list = df_green.to_dict(orient='records')

# Send data to the topic
for data in data_list:
    # Include column names in the data
    data_with_columns = {col: data[col] for col in df_green.columns}
    producer.send(topic_name, value=data_with_columns)

# Flush the producer
producer.flush()
# Record the end time
end_time = time.time()

# Calculate the time taken
processing_time = end_time - start_time
print(f'Time taken: {processing_time:.2f} seconds')

Time taken: 61.12 seconds


In [ ]:
#!docker exec -it redpanda-1 rpk topic consume green-trips
#!docker exec -it redpanda-1 rpk topic consume green-trips --offset 0



In [10]:
pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

24/03/24 18:50:12 WARN Utils: Your hostname, localhost.localdomain resolves to a loopback address: 127.0.0.1; using 192.168.203.168 instead (on interface ens33)
24/03/24 18:50:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/zoomcamp/spark/spark-3.4.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1d5209ec-dff7-46fa-9279-38654c3272b6;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.2 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 727ms :: artifacts dl 13ms
	:: modules in us

In [13]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [14]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        #print(first_row[0])
        display(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()





24/03/24 18:50:33 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-5e3f31fa-685b-4388-9198-f6bf46b77a4c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/24 18:50:33 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/03/24 18:50:34 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


Row(key=None, value=bytearray(b'{"lpep_pickup_datetime": "2019-10-01 00:26:02", "lpep_dropoff_datetime": "2019-10-01 00:39:58", "PULocationID": 112, "DOLocationID": 196, "passenger_count": 1.0, "trip_distance": 5.88, "tip_amount": 0.0}'), topic='green-trips', partition=0, offset=0, timestamp=datetime.datetime(2024, 3, 24, 18, 45, 39, 331000), timestampType=0)

query.stop()

In [15]:


schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [16]:
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [17]:
# Start the streaming query and display the results
query = green_stream \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

# Wait for the stream to finish
query.awaitTermination()

24/03/24 18:51:00 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-7fc7d669-14c7-48fd-9c24-839bcbd0d72d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/24 18:51:00 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/03/24 18:51:00 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+---------------------+------------+------------+---------------+-------------+----------+
|lpep_pickup_datetime|lpep_dropoff_datetime|PULocationID|DOLocationID|passenger_count|trip_distance|tip_amount|
+--------------------+---------------------+------------+------------+---------------+-------------+----------+
| 2019-10-01 00:26:02|  2019-10-01 00:39:58|         112|         196|            1.0|         5.88|       0.0|
| 2019-10-01 00:18:11|  2019-10-01 00:22:38|          43|         263|            1.0|          0.8|       0.0|
| 2019-10-01 00:09:31|  2019-10-01 00:24:47|         255|         228|            2.0|          7.5|       0.0|
| 2019-10-01 00:37:40|  2019-10-01 00:41:49|         181|         181|            1.0|          0.9|       0.0|
| 2019-10-01 00:08:13|  2019-10-01 00:17:56|          97|         188|            1.0|         2.52|      2.26|
| 2019-

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/zoomcamp/spark/spark-3.4.2-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/zoomcamp/spark/spark-3.4.2-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/root/anaconda3/envs/production/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [18]:
if query.isActive:
    query.stop()


# Add a timestamp column
df_with_timestamp = green_stream.withColumn("timestamp", F.current_timestamp())

# Group by a 5-minute window and destination location ID, then count occurrences
popular_destinations = df_with_timestamp \
    .groupBy(F.window("timestamp", "5 minutes"), "DOLocationID") \
    .count() \
    .orderBy(F.col("count").desc())

# Start the query to output to console
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

# Await termination of the query
query.awaitTermination()

24/03/24 18:57:19 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-31be51f3-4b67-4ade-a488-cad80cb82ec6. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/24 18:57:19 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/03/24 18:57:19 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-24 18:55:00, 2024-03-24 19:00:00}|74          |17741|
|{2024-03-24 18:55:00, 2024-03-24 19:00:00}|42          |15942|
|{2024-03-24 18:55:00, 2024-03-24 19:00:00}|41          |14061|
|{2024-03-24 18:55:00, 2024-03-24 19:00:00}|75          |12840|
|{2024-03-24 18:55:00, 2024-03-24 19:00:00}|129         |11930|
|{2024-03-24 18:55:00, 2024-03-24 19:00:00}|7           |11533|
|{2024-03-24 18:55:00, 2024-03-24 19:00:00}|166         |10845|
|{2024-03-24 18:55:00, 2024-03-24 19:00:00}|236         |7913 |
|{2024-03-24 18:55:00, 2024-03-24 19:00:00}|223         |7542 |
|{2024-03-24 18:55:00, 2024-03-24 19:00:00}|238         |7318 |
|{2024-03-24 18:55:00, 2024-03-24 19:00:00}|82          |7292 |
|{2024-

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/zoomcamp/spark/spark-3.4.2-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/zoomcamp/spark/spark-3.4.2-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/root/anaconda3/envs/production/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 